In [1]:
library('trendsceek')
library("dplyr")
library('Seurat')
library('data.table')


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: ggplot2
Loading required package: cowplot

Attaching package: ‘cowplot’

The following object is masked from ‘package:ggplot2’:

    ggsave

Loading required package: Matrix
Warning message:
“package ‘data.table’ was built under R version 3.4.4”
Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [2]:
winsorize <- function(x, n.win = 1){
###Winsorize extreme values
###Ex: Two most extreme values (from each side):
###t(apply(ed, 1, winsorize, 2))

    n.vals = length(x)
    fraction = n.win / n.vals
    if(length(fraction) != 1 || fraction < 0 || fraction > 0.5){
        stop("bad value for 'fraction'")
    }

    win.sorted.ind = order(x)
    x[win.sorted.ind[1:n.win]] = x[win.sorted.ind[n.win + 1]]
    x[win.sorted.ind[(n.vals - n.win + 1):n.vals]] = x[win.sorted.ind[n.vals - n.win]]

    ##fraction-based
    if(0){
        lim = stats::quantile(x, probs = c(fraction, 1 - fraction))

        ##set extreme values
        x[ x < lim[1] ] = lim[1]
        x[ x > lim[2] ] = lim[2]
    }
    
    return(x)
}

In [2]:
df='../Data/middle_exp_mat.csv'
data=read.csv(df,header = T,row.names=1)
counts=t(data)

In [4]:
win.counts=t(apply(counts,1,winsorize,4))


Rep<-genefilter_exprmat(counts,min.expr=5,min.ncells.expr = 3)
Rep <-CreateSeuratObject(raw.data = Rep ,min.cells = 3,min.genes = 5 )
Rep_filt<- FilterCells(Rep, subset.names = "nGene", low.thresholds = 500, high.thresholds = Inf)
n.gene=nrow(Rep_filt@raw.data)
n.cell=ncol(Rep_filt@raw.data)
Rep<-NormalizeData(Rep_filt,normalization.method = "LogNormalize",scale.factor = 10000)

In [5]:
info<-read.csv("../Data/middle_sample_info.csv",header = T,row.names = 1)
corr<-data.frame(x=info$abs_X,y=info$abs_Y)

In [6]:
head(corr)

x,y
-438.9991,-312.4758
-437.4286,-334.3169
-432.6566,-438.8867
-429.7818,-411.2898
-412.9161,-358.0686
-410.7389,-310.5912


In [7]:
pp=pos2pp(corr)
pp=set_marks(pp,win.counts,log.fcn = log10,pseudo.count=1)
sub_pp=head(colnames(pp$marks),10)
pp2plot=pp_select(pp,sub_pp)

In [ ]:
#t0<-Sys.time()
trendstat_list=trendsceek_test(pp,ncores = 1,nrand = 10000)
#t1<-Sys.time()
# t=t1-t0

In [ ]:
sig_list = extract_sig_genes(trendstat_list, alpha=0.05)
lapply(sig_list, nrow)

sig_gene_v<-sig_list$markvario$gene

sig_gene_c<-sig_list$markcorr$gene

sig_genes=union(sig_gene_v,sig_gene_c)

df_sig=data.frame(sig_gene=sig_genes)

write.csv(df_sig,'results/merfish_sig_list.csv')